<a href="https://colab.research.google.com/github/rfasich24/Skripsi-E-Nose/blob/main/Mencari_Nilai_RO_dari_Masing_Masing_Sensor_per_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mencari nilai RO

In [2]:
import pandas as pd
import glob
import numpy as np
import os # Ditambahkan untuk mengambil nama file

# --- KONFIGURASI PENGGUNA (SESUAI INPUT ANDA) ---

# 1. Path file CSV di runtime.
#    "*.csv" berarti "semua file .csv di direktori kerja saat ini"
#    Ini adalah tempat file Anda akan muncul setelah di-upload.
FOLDER_PATH = "*.csv"
OUTPUT_FILENAME = 'hasil_ro.csv' # Nama file untuk menyimpan hasil

# 2. Nilai Resistor Beban (RL) dalam Ohm (default 10 Ohm)
LOAD_RESISTORS = {
    'MQ2': 7.0,
    'MQ3': 2.0,
    'MQ4': 1.0,
    'MQ7': 1.0,
    'MQ8': 1.0,
    'MQ135': 1.0,
    'MQ137': 1.0,
    'TGS': 1.0
}

# 3. Rentang indeks/baris untuk periode udara bersih (Counter 800-1000)
CLEAN_AIR_START_INDEX = 800
CLEAN_AIR_END_INDEX = 1001  # (Mengambil baris 800 s/d 1000)

# 4. Parameter ADC dari ESP32
ADC_RESOLUTION = 4095  # 12-bit

# ----------------------------------------------------------------

# Fungsi untuk menghitung resistansi sensor (Rs) dari nilai ADC
def calculate_rs(adc_value, rl_value):
    """Menghitung resistansi sensor (Rs) dari nilai ADC."""
    if adc_value == 0:
        return np.inf
    return rl_value * (ADC_RESOLUTION / adc_value - 1)

# List nama kolom sensor
sensor_columns = list(LOAD_RESISTORS.keys())

# Dictionary untuk menyimpan semua hasil R0
all_ro_results = {}

# Mencari semua file CSV di path yang ditentukan
all_files = glob.glob(FOLDER_PATH)

# --- PERUBAHAN LOGIC UNTUK IPYNB ---
# Filter daftar file agar tidak menyertakan file outputnya sendiri,
# untuk mencegah error jika sel ini dijalankan berulang kali.
csv_files = [f for f in all_files if f != OUTPUT_FILENAME]
# -------------------------------------

if not csv_files:
    print(f"Error: Tidak ada file .csv yang ditemukan di runtime.")
    print("Pastikan Anda sudah meng-upload 25 file CSV Anda.")
else:
    print(f"Ditemukan {len(csv_files)} file CSV. Memulai proses...")

    for file_path in csv_files:
        try:
            # Mengambil nama file saja
            file_name = os.path.basename(file_path)

            df = pd.read_csv(file_path)

            missing_sensors = []
            for sensor in sensor_columns:
                if sensor not in df.columns:
                    missing_sensors.append(sensor)

            if missing_sensors:
                print(f"Peringatan di {file_name}: Kolom {missing_sensors} tidak ditemukan.")

            clean_air_df = df.iloc[CLEAN_AIR_START_INDEX:CLEAN_AIR_END_INDEX]

            if clean_air_df.empty:
                print(f"Peringatan di {file_name}: Tidak ada data pada rentang indeks {CLEAN_AIR_START_INDEX}-{CLEAN_AIR_END_INDEX}.")
                continue

            ro_per_file = {}

            for sensor in sensor_columns:
                if sensor in df.columns:
                    avg_adc_clean_air = clean_air_df[sensor].mean()

                    if pd.isna(avg_adc_clean_air):
                        print(f"Peringatan di {file_name} untuk {sensor}: Data 'udara bersih' kosong atau NaN.")
                        ro_per_file[sensor] = np.nan
                    else:
                        ro_value = calculate_rs(avg_adc_clean_air, LOAD_RESISTORS[sensor])
                        ro_per_file[sensor] = ro_value

            all_ro_results[file_name] = ro_per_file

        except Exception as e:
            print(f"Gagal memproses file {file_path}: {e}")

    if all_ro_results:
        results_df = pd.DataFrame.from_dict(all_ro_results, orient='index')
        # Membulatkan nilai R0 menjadi 5 angka di belakang koma
        results_df = results_df.round(5)
        results_df.to_csv(OUTPUT_FILENAME)

        print("\n--- Proses Selesai ---")
        print("Nilai R0 rata-rata untuk setiap sensor per sampel (dalam Ohm):")
        print(results_df)
        print(f"\nHasil telah berhasil disimpan ke file '{OUTPUT_FILENAME}'")
        print("Anda dapat men-download file tersebut dari file browser di sebelah kiri.")

Ditemukan 20 file CSV. Memulai proses...

--- Proses Selesai ---
Nilai R0 rata-rata untuk setiap sensor per sampel (dalam Ohm):
                                         MQ2       MQ3       MQ4      MQ7  \
SPLIT_0_1000_Sample 5 Grade 2.csv  224.75401  11.44737  11.21603  8.03053   
SPLIT_0_1000_Sample 1 Grade 2.csv  243.36566  10.07743  13.68239  8.41218   
SPLIT_0_1000_Sample 1 Grade 3.csv  117.52538  11.72797  11.29656  7.47887   
SPLIT_0_1000_Sample 3 Grade 2.csv  215.69026  11.81009  12.93069  8.16373   
SPLIT_0_1000_Sample 4 Grade 3.csv  336.38548  12.36704  12.64887  8.50248   
SPLIT_0_1000_Sample 2 Grade 3.csv  141.02346  11.47150  11.94521  7.92641   
SPLIT_0_1000_Sample 4 Grade 4.csv  202.19402  12.01813  11.18111  6.66452   
SPLIT_0_1000_Sample 2 Grade 5.csv  186.98892  12.83375  10.84343  7.54214   
SPLIT_0_1000_Sample 2 Grade 2.csv  153.78766  11.12678  13.30027  7.17211   
SPLIT_0_1000_Sample 3 Grade 3.csv  277.09176  12.73998  12.45278  8.20050   
SPLIT_0_1000_Sample 3 Gra